# MetaboLights to CMAP
## Krista Longnecker, 27 June 2025


MetaboLights has FTP access to their data files and that is easy enough to access, but there are some downstream steps to add because I did not upload the full station inforamtion to MetaboLights.

In [4]:
import pandas as pd
from ftplib import FTP
#from openpyxl import Workbook
#pd.options.mode.copy_on_write = True #will be the default, may as well set it now

In [5]:
# start with the TSQ data (only other dataset ready is the untargeted data)
study_id = 'MTBLS2356'

In [6]:
ftp = FTP('ftp.ebi.ac.uk') #address from MetaboLights webpage
ftp.login()
ftp.cwd('/pub/databases/metabolights/studies/public/' + study_id)
#ftp.retrlines('LIST') #this will only print to console, not what I want
fileList = ftp.nlst() #can use this to make a list that will be searchable

In [7]:
fileList

['FILES',
 'HASHES',
 'METADATA_REVISIONS',
 'a_MTBLS2356_LC-MS_negative__metabolite_profiling.txt',
 'a_MTBLS2356_LC-MS_positive__metabolite_profiling.txt',
 'i_Investigation.txt',
 'm_MTBLS2356_LC-MS_negative__metabolite_profiling_v2_maf.tsv',
 'm_MTBLS2356_LC-MS_positive__metabolite_profiling_v2_maf.tsv',
 's_MTBLS2356.txt']

In [8]:
#start with the metadata information so I can convert each sample to time/lat/lon/depth to match the CMAP requirements
str = 's_' + study_id #this is the search string for the data files
metadataFiles = [v for v in fileList if str in v] 
metadataFiles = pd.DataFrame(metadataFiles,columns = ['files'])

# metadataFiles
writeFile = 'data/' + 'tempMetadata.txt'
readFile = metadataFiles.loc[0,'files']

with open(writeFile,'wb') as fp:
    try:
        retr_command = f"RETR {readFile}"
        ftp.retrbinary(retr_command, fp.write)
    except Exception as e: 
        print(f"Error during quit: {e}")
    except AttributeError as e: 
        print(f"AttributeError during quit: {e} - connection was likely already closed.")
    
#ftp.quit()  #can close this down now as I have the files I need (careful when I get to the loop
#print("FTP closed")

metadata = pd.read_table(writeFile,delimiter = '\t')

In [9]:
#pull Source Name as I need that later to match to columns in the data file
sampleNames  = metadata['Source Name']
depth = metadata['Factor Value[Depth]']
#time is messier and the MetaboLights columns names are long, so shorten them to make this easier
temp = metadata[['Factor Value[Sampling year date]','Factor Value[Sampling month date]',
                 'Factor Value[Sampling day date]','Factor Value[Hour of the day]','Factor Value[Minute of the hour]']]
temp.columns = ['year','month','day','hour','minute']

In [10]:
step1 = pd.to_datetime(dict(year=temp.year,month=temp.month,day = temp.day,hour = temp.hour,minute=temp.minute))
date_cmap = step1.dt.strftime("%Y-%m-%dT%H:%M:%S")
date_cmap.head()

0    2016-07-09T18:04:00
1    2016-07-09T18:04:00
2    2016-07-09T18:04:00
3    2016-07-10T00:02:00
4    2016-07-10T00:02:00
dtype: object

### Need BIOS-SCOPE file for lat/lon information

In [11]:
#will need the BIOS-SCOPE discrete data file for station information - that will have both BATS and BIOS-SCOPE data in it
fName = 'data/BATS_BS_COMBINED_MASTER_latest.xlsx';
BSdata = pd.read_excel(open(fName,'rb'),sheet_name = 'DATA')

In [12]:
#MetaboLights required samples to begin with a letter, I used 's' and need to strip that out 
NewID_inMTBLS  = pd.to_numeric(sampleNames.str.strip('s')) 
#convert the series into a dataframe:
s_df = NewID_inMTBLS.reset_index()

#use merge as it will be sorted in the right order
merged_df = pd.merge(BSdata,s_df,how='right',left_on='New_ID',right_on='Source Name')

# Data

In [13]:
# Now get the data files (more than one because things are split positive/negative ion mode...concatenate them later
str = 'm_' + study_id #this is the search string for the data files
dataFiles = [v for v in fileList if str in v] #Python syntax, will make a list
dataFiles = pd.DataFrame(dataFiles,columns = ['files']) #I find the dataframe easier to manage than the list

In [14]:
idx = 0 #make a loop later as can have multiple data files for a single dataset
writeFile = 'data/' + 'tempData.tsv'
readFile = dataFiles.loc[0,'files']

In [15]:
#while testing, if the FTP command fails the connection is left open and the next command gives error
#error is: AttributeError: 'NoneType' object has no attribute 'sendall'
with open(writeFile,'wb') as fp:
    #try-except to make sure the FTP closes
    try:
        retr_command = f"RETR {readFile}"
        ftp.retrbinary(retr_command, fp.write)
    except Exception as e: 
        print(f"Error during quit: {e}")
        
ftp.quit()  #close the FTP connection

'221 Goodbye.'

In [16]:
#now the test, can I read in the resulting file? YES, so this is the datafile
tsvFile = pd.read_table(writeFile,delimiter = '\t')

In [23]:
tsvFile.head

<bound method NDFrame.head of    database_identifier chemical_formula  \
0          CHEBI:18026           C7H6O4   
1          CHEBI:77267          C3H8O5S   
2          CHEBI:44111          C3H6O2S   
3          CHEBI:30763           C7H6O3   
4          CHEBI:27744         C3H8NO5P   
5          CHEBI:16750       C10H13N5O5   
6          CHEBI:17596       C10H12N4O5   
7           CHEBI:6650           C4H6O5   
8          CHEBI:17533         C7H11NO5   
9          CHEBI:47571        C11H19NO8   
10         CHEBI:68329          C9H10O5   
11         CHEBI:28865       C26H45NO7S   
12         CHEBI:17748       C10H14N2O5   
13         CHEBI:18107       C10H12N4O6   

                                               smiles  \
0                                  OC(=O)c1cccc(O)c1O   
1                                   OCC(O)CS(O)(=O)=O   
2                                           OC(=O)CCS   
3                                   OC(=O)c1ccc(O)cc1   
4                                  OC(=

In [42]:
#column with metabolite name is  (database identifier would be more generic, need to talk to CMAP people about this)
mtabColumn = 'database_identifier'
# mtabColumn = 'metabolite_identification' 

#only keep the columns that are in sampleNames
dataColumns = tsvFile.columns[tsvFile.columns.isin(sampleNames)]
dataOnly = tsvFile.loc[:,dataColumns].transpose() #index is the 's' numbered samples

dataOnly.columns = tsvFile[mtabColumn] #label the columns with the metabolite information, will also use this for the sheet with metadata about the variables
nVariables = len(dataOnly.columns) #need this for the sheet for the metadata on the variables

In [28]:
#start assembling into CMAP format
# Required variables are time, lat, lon, depth
df = pd.DataFrame(columns=['time','lat','lon','depth'])
df['time'] = date_cmap.to_frame()
df['depth'] = depth.to_frame()
df['lat'] = merged_df['latN'].to_frame()
df['lon'] = -merged_df['lonW'].to_frame() #need negative number to put this into -180 to 180 space
#df.insert(1,'test',merged_df['New_ID']) #check that I have the indexing right
#df.insert(1,'test2',s_df['Source Name'])
df.insert(1,'forIndex',sampleNames) #need an index to keep the rows matched up
df.set_index('forIndex',inplace=True)

#concatenate with the data in dataOnly
df = pd.concat([df,dataOnly],axis=1,ignore_index = False)

metadata about the variables

In [33]:
# work on the second sheet: metadata about the variables; use the CMAP dataset template to setup the dataframe so I get the column headers right
fName = 'datasetTemplate.xlsx'
sheet_name = 'vars_meta_data'
vars = pd.read_excel(fName, sheet_name=sheet_name)
cols = vars.columns.tolist()
#df2 will be the dataframe with the metadata about the variables, set it up empty here
df2 = pd.DataFrame(columns=cols,index = pd.RangeIndex(0,nVariables,1))

In [50]:
#need to details about the metabolites

# this is only a partial list of variables for the moment
df2['var_short_name'] = dataOnly.columns
df2.loc[:,'var_long_name'] = tsvFile.loc[:,'metabolite_identification']
df2.loc[:,'var_sensor'] = 'Triple quadrupole mass spectrometer (TSQ Vantage, Thermo Scientific)'
df2.loc[:,'var_unit'] = 'pM' #this is in the protocols, but I also have some inside information here
df2.loc[:,('var_spatial_res')] = 'irregular'
df2.loc[:, ('var_temporal_res')] = 'irregular'
df2.loc[:,('var_discipline')] = 'chemistry'
df2.loc[:,('var_visualize')] = 1 #yes/no, all metabolites can be visualized

In [ ]:
#'var_keywords' will be the hardest as the metabolites have many, many keywords. 
#I want to talk to the people at CMAP about best options to handle that (start a list of topics)

metadata about the project

In [ ]:
## update this ... can use information from MetaboLights (later)

# finally gather up the dataset_meta_data
# assemble the details here, might setup in a separate text file later
df3 = pd.DataFrame({
    'dataset_short_name': ['BIOSSCOPE_v1'],
    'dataset_long_name': ['BIOS-SCOPE discrete sample data'],
    'dataset_version': ['1.0'],
    'dataset_release_date': ['2025-06-25'],
    'dataset_make': ['observation'],
    'dataset_source': ['Craig Carlson, Bermuda Institute of Ocean Sciences'],
    'dataset_distributor': ['Craig Carlson, Bermuda Institute of Ocean Sciences'],
    'dataset_acknowledgement': ['We thank the BIOS-SCOPE project team and the BATS team for assistance with sample collection, processing, and analysis. The efforts of the captains, crew, and marine technicians of the R/V Atlantic Explorer are a key aspect of the success of this project. This work supported by funding from the Simons Foundation International.'],
    'dataset_history': [''],
    'dataset_description': ['This dataset includes analyses from Niskin bottle samples collected on R/V Atlantic Explorer cruises as part of the BIOS-SCOPE campaign in the time period from 2016 until 2025. Included are CTD data, and survey biogeochemical samples including inorganic nutrients, particulate organic carbon and nitrogen, dissolved organic carbon, dissolved organic nitrogen, total dissolved amino acids, bacterial abundance and production.'],
    'dataset_references': ['Carlson, C. A., Giovannoni, S., Liu, S., Halewood, E. (2025) BIOS-SCOPE survey biogeochemical data as collected on Atlantic Explorer cruises (AE1614, AE1712, AE1819, AE1916) from 2016 through 2019. Biological and Chemical Oceanography Data Management Office (BCO-DMO). (Version 1) Version Date 2021-10-17. doi:10.26008/1912/bco-dmo.861266.1 [25 June 2025]'],
    'climatology': [0]
    })

#get the list of cruise names from the bcodmo data file
t = pd.DataFrame(bcodmo['Cruise_ID'].unique())
t.columns = ['cruise_names']
df3 = pd.concat([df3,t],axis=1,ignore_index = True)



In [ ]:
fName_CMAP = 'data/forCMAPfromMetabolights.xlsx'
dataset_names = {'data': df, 'dataset_meta_data': df3, 'vars_meta_data': df2}
with pd.ExcelWriter(fName_CMAP) as writer:
    for sheet_name, data in dataset_names.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
raise UserWarning('Stopping and leave code below for historical reasons, code will not run')

In [ ]:
#If I can get isatools to install, this will be an easier way (I think/hope) to access data at BCO-DMO. There is also an R equivalent at metabolighteR that provides access to MetaboLights REST API

In [ ]:
#pip install isatools #fails with this error:AttributeError: module 'pkgutil' has no attribute 'ImpImporter'. Did you mean: 'zipimporter'?
pip install git+https://github.com/ISA-tools/isa-api/

In [ ]:
import isatools

In [ ]:
from isatools.net import mtbls as MTBLS
tmp_dir = MTBLS.get('MTBLS2356')